<a href="https://colab.research.google.com/github/alisonsoong/NASA-SEES-Internship-2021/blob/main/Adapted_from_Geemap_ee_map_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_and_earthengine.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Import Earth Engine API and authenticate<a class="anchor" id="import-api"></a>

The Earth Engine API is installed by default in Google Colaboratory. The following steps must be completed

*   for each new Colab session,
*   if you restart your Colab kernel, or 
*   if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the Google Earth Engine API into your session.

In [1]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [2]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=83b7Y4lV3zntavACGJLr5A1Bfg1JUGFlq2SkVgvvYQ4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi1RbJQRJFqYfeptJ7ey12xBtcx95x4pJ0Rv6MqPUJhenDKBwX3EiU

Successfully saved authorization token.


## Install the geemap Python package

Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api and folium for map display. One dependency not installed here is ipyleaflet, since Colab sadly does not support it.

With **geemap** installed, we do not have to write our own **folium** display function.

In [3]:
# Installs geemap package
import subprocess

try:
    import geemap
    print("geemap is imported and ready to use in Colab")
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap
    print("geemap is now installed, imported and ready to use in Colab") 


geemap package not installed. Installing ...
geemap is now installed, imported and ready to use in Colab


# Basics Functions in GEE Colab & geemap Activity

Here the Javascript code (link on slide 32 of the ARSET Session 3) is converted into Python.

[Link to Presentation Slides PDF](https://appliedsciences.nasa.gov/sites/default/files/2021-06/GEE_Land_Part3.pdf)

From ARSET training **Part 3: Google Earth Engine Basics and General Applications**
*Wednesday, June 16, 2021*




In [29]:
Map = geemap.Map()

## Define a function
This is a special function to be used in Google Earth engine on the server side.

In [4]:
def minus1000(num):
    result = ee.Number(num).subtract(1000)
    return result

In [6]:
result = minus1000(2000)
print(result.getInfo())

1000


## Apply the map function

In [7]:
years = ee.List.sequence(2000, 2010)
print(years.getInfo())

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [8]:
results = years.map(minus1000)
print(results.getInfo())

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]


## Use anonymous/lambda function

In [11]:
years = ee.List.sequence(2000, 2010)

results = years.map(lambda num: ee.Number(num).subtract(1000)) # exactly the same as above
print(results.getInfo())

[1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]


## Filter an ImageCollection

In [74]:
roi = ee.Geometry.Point(-83.92, 35.96)  # Knoxville, Tennessee

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate("2019-01-01", "2020-01-01") \
    .sort("CLOUD_COVER")

collectionNight1  = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
    .filterBounds(roi) \
    .filterDate("2019-01-01", "2020-01-01") \

collectionNight2  = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
    .filterBounds(roi) \
    .filterDate("2020-01-01", "2021-01-01") \

print(collection.size().getInfo()) # to query how many images you are getting back (landsat8 starts in 2013)

17


In [75]:
image = collection.first() # sorted by cloud cover!
imageNight1 = collectionNight1.first() # sorted by cloud cover!
imageNight2 = collectionNight2.first() # sorted by cloud cover!

In [16]:
image.getInfo()

{'bands': [{'crs': 'EPSG:32617',
   'crs_transform': [30, 0, 121185, 0, -30, 4111815],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7851, 7971],
   'id': 'B1'},
  {'crs': 'EPSG:32617',
   'crs_transform': [30, 0, 121185, 0, -30, 4111815],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7851, 7971],
   'id': 'B2'},
  {'crs': 'EPSG:32617',
   'crs_transform': [30, 0, 121185, 0, -30, 4111815],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7851, 7971],
   'id': 'B3'},
  {'crs': 'EPSG:32617',
   'crs_transform': [30, 0, 121185, 0, -30, 4111815],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7851, 7971],
   'id': 'B4'},
  {'crs': 'EPSG:32617',
   'crs_transform': [30, 0, 121185, 0, -30, 4111815],
   'data_type

In [27]:
test_dict = geemap.image_props(image).getInfo() # a dictionary
test_time = geemap.image_props(image).getInfo()['SENSING_TIME']
print(test_time) # or
print(test_dict['SENSING_TIME'])
print(test_dict['IMAGE_DATE'])

2019-03-17T16:11:33.5001919Z
2019-03-17T16:11:33.5001919Z
2019-03-17


In [18]:
collection.aggregate_array("CLOUD_COVER").getInfo() # the cloud cover of the images in the collection

[0.06,
 0.42,
 0.57,
 0.76,
 1.26,
 6.67,
 9.69,
 13.14,
 22.55,
 26.67,
 29.89,
 33.11,
 51.76,
 74.72,
 78.86,
 85.73,
 95.02]

In [76]:
vis_params = {
    'bands': ['B5', 'B4', 'B3'], # B5 = NIR (highly reflective of green vegetation), B4 = red, B3 = green
    # 'bands': ['B6', 'B5', 'B3'], # a nice false color combination (water is a dark blue)
    'min': 0, 
    'max': 3000,
    'gamma': 1.0
}

night_light_params = {
    'bands': ['avg_rad'],
    'min': 0, 
    'max': 30,
    'gamma': 1.0
}

Map = geemap.Map()
Map.addLayer(image, vis_params, "First image")
Map.addLayer(imageNight1, night_light_params, "Night image: 2019")
Map.addLayer(imageNight2, night_light_params, "Night image: 2020")
Map.centerObject(image)
Map


## Select the best image each year

In [34]:
roi = ee.Geometry.Point(-83.92, 35.96)  # Knoxville, Tennessee


def best_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    
    image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .sort("CLOUD_COVER") \
        .first() # first image, which is the best image with the least cloud cover
    
    return image

In [35]:
start_year = 2013
end_year = 2020
years = ee.List.sequence(start_year, end_year)
year_list = years.getInfo()
print(year_list)

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [36]:
images = years.map(best_image)

In [37]:
count = images.size().getInfo()
print(count)

8


In [38]:
ee.ImageCollection(images).aggregate_array("CLOUD_COVER").getInfo()

[0.09, 0.13, 0.03, 0.01, 0.03, 0.01, 0.06, 4.61]

In [46]:
Map = geemap.Map()
for index in range(0, count):
    image = ee.Image(images.get(index))
    layer_name = "Image " + str(year_list[index])
    Map.addLayer(image, vis_params, layer_name, False)

In [47]:
Map.centerObject(roi)
Map

## Set image properties

In [48]:
collection = ee.ImageCollection(images)

In [49]:
collection.aggregate_array("system:time_start").getInfo()

[1382804022390,
 1399392695740,
 1442247113710,
 1454688719630,
 1494778283540,
 1525191067880,
 1552839093500,
 1591546292340]

In [50]:
collection = collection.map(lambda img: img.set({"DATE": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")}))

In [51]:
collection.aggregate_array("DATE").getInfo()

['2013-10-26',
 '2014-05-06',
 '2015-09-14',
 '2016-02-05',
 '2017-05-14',
 '2018-05-01',
 '2019-03-17',
 '2020-06-07']